In [0]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta1-table")

In [0]:
df = spark.read.format("delta").load("/tmp/delta1-table")
df.show()

+---+
| id|
+---+
|  3|
|  4|
|  0|
|  1|
|  2|
+---+



In [0]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta1-table")

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

In [0]:
deltaTable = DeltaTable.forPath(spark, "/tmp/delta1-table")

In [0]:

deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })
deltaTable.toDF().show()

+---+
| id|
+---+
|  5|
|  7|
|  9|
|806|
|808|
+---+



In [0]:
deltaTable.delete(condition = expr("id % 2 == 0"))
deltaTable.toDF().show()

+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+



In [0]:
# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+

